# Libraries

In [16]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.cluster import KMeans
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.layers import BatchNormalization, Input, Dense
from IPython.display import Image, display


# Dataset


In [ ]:
url_resep = 'https://docs.google.com/spreadsheets/d/1pMk5X9NjS-m5JWWDjF3G6QT9-LUmcs6Pl-vNfVpWJbc/export?format=csv'
df_resep = pd.read_csv(url_resep)

url_train = 'https://docs.google.com/spreadsheets/d/1_eocx0nxgA6pu59z515kW9M9PUULs7GcjHLX6aFY8aY/export?format=csv'
df_train = pd.read_csv(url_train)

# Pre-Processing

In [ ]:
df_resep.head()

,nama_makanan,kalori,bahan1,jumlah1,bahan2,jumlah2,bahan3,jumlah3,bahan4,jumlah4,...,jumlah14,bahan15,jumlah15,cara_masak,image,porsi,protein_per_porsi,karbohidrat_per_porsi,jenis,cluster
0,overnight oat,380.0,oatmeal,50 gram,susu kedelai,200 ml,strawberry,2 buah,chia seed,secukupnya,...,NaN,NaN,NaN,1. Tuang oatmeal ke wadah kaca atau kotak maka...,https://img-global.cpcdn.com/recipes/321becdb0...,1,16.50,57.54,pokok,0
1,telur super sehat,282.0,telur,3 butir,wortel,1 buah,daun bawang,3 batang,garam,secukupnya,...,NaN,NaN,NaN,1. Aduk semua bahan jadi satu. 2. Siapkan waja...,https://img-global.cpcdn.com/recipes/1503c362b...,1,19.08,10.09,lauk,3
2,sup rumput laut,130.0,pasta tauco jepang,1 sdm,kaldu ikan,1 sdt,tofu,1 bungkus,daun bawang,1 batang,...,NaN,NaN,NaN,"1.Siapkan mangkuk, rendam rumput laut kering d...",https://img-global.cpcdn.com/recipes/ba94ecdb6...,1,10.50,5.83,sayuran,2
3,sayur bayam kecambah,50.0,bayam,1/2 ikat,kecambah,secukupnya,bawang merah,2 siung,bawang putih,2 siung,...,NaN,NaN,NaN,"1. tumis bawang merah, bawang putih, dan cabai...",https://img-global.cpcdn.com/recipes/53b582ddb...,1,5.60,8.36,sayuran,2
4,tumis terung goreng telur,278.0,terung,1 buah,telur,3 butir,kecap asin,3 sdm,cabai,3 biji,...,NaN,NaN,NaN,"1. Iris tipis terung, goreng dengan minyak sec...",https://img-global.cpcdn.com/recipes/76e3b86e0...,1,22.90,18.00,sayuran,3


In [ ]:
df_train.head()

,Gender,Height,Weight,Age,Activity,Target,BMR,BMR X ACT,DEFISIT,KEBUTUHAN,Cluster
0,1,174,96,76,1.375,-2.0,1778.048,2444.816000,-1100,1344.816000,4.0
1,1,189,87,35,1.725,-2.0,1962.217,3384.824325,-1100,2284.824325,4.0
2,0,185,110,24,1.200,-4.0,1933.973,2320.767600,-2200,120.767600,2.0
3,0,195,104,68,1.375,-1.0,1718.951,2363.557625,-550,1813.557625,4.0
4,1,149,61,68,1.900,-3.0,1234.594,2345.728600,-1650,695.728600,4.0


# Model

In [ ]:
# call back
class MyThresholdCallback(tf.keras.callbacks.Callback):
    def __init__(self, threshold):
        super(MyThresholdCallback, self).__init__()
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None):
        val_acc = logs["accuracy"]
        if val_acc >= self.threshold:
            self.model.stop_training = True

# Pisahkan fitur dan label
X = df_train[['Weight', 'Height', 'Age', 'Gender', 'Activity', 'Target']].values
y = df_train['Cluster'].values

# One-hot encoding untuk label
# 'Cluster' column has values from 0 to 4 (5 unique clusters)

encoder = OneHotEncoder(sparse_output=False, categories='auto')  # categories='auto' to infer categories from data
y_encoded = encoder.fit_transform(y.reshape(-1, 1))

# Periksa jumlah kelas unik dalam data Anda
num_classes = y_encoded.shape[1]

model = Sequential([
    Input(shape=(6,), dtype=tf.float32),
    Dense(64, activation='relu', input_shape=(6,)),
    BatchNormalization(),
    Dense(32, activation='relu'),
    BatchNormalization(),
    Dense(16, activation='relu'),
    Dense(num_classes, activation='softmax')
])
model.compile(optimizer='RMSprop', loss='categorical_crossentropy', metrics=['accuracy'])

# Split data untuk training dan validation
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

#early stopping
my_callback = MyThresholdCallback(threshold=0.94)  # Set the desired accuracy threshold

# Training model with data and callback
model.fit(X_train, y_train, epochs=300, batch_size=32,
          validation_data=(X_val, y_val), callbacks=[my_callback])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 9s 121ms/step - accuracy: 0.3494 - loss: 1.5120 - val_accuracy: 0.0490 - val_loss: nan
Epoch 2/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6403 - loss: 1.1549 - val_accuracy: 0.0490 - val_loss: nan
Epoch 3/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7251 - loss: 0.9692 - val_accuracy: 0.0686 - val_loss: nan
Epoch 4/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7323 - loss: 0.9231 - val_accuracy: 0.0686 - val_loss: nan
Epoch 5/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7075 - loss: 0.9435 - val_accuracy: 0.0784 - val_loss: nan
Epoch 6/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.7592 - loss: 0.8239 - val_accuracy: 0.3235 - val_loss: nan
Epoch 7/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.8010 - loss: 0.7561 - val_accuracy: 0.5588 - val_loss: nan
Epoch 8/300
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.7679 - loss: 0.7108 - val_accuracy: 0.6961 - val_loss: nan
Ep

# Prediction


In [ ]:
# Meminta input pengguna
bb = float(input("Masukkan berat badan (kg): "))
tinggi_badan = float(input("Masukkan tinggi badan (cm): "))
umur = int(input("Masukkan umur (tahun): "))
gender = int(input("Masukkan gender (0 untuk perempuan, 1 untuk laki-laki): "))
tingkat_aktivitas = float(input("Masukkan tingkat aktivitas (contoh: 1.375 untuk aktif ringan): "))
target_bb = float(input("Masukkan target penurunan berat badan (misalnya: -4 untuk turun 4 kg): "))

# Membuat array baru berdasarkan input
new_user_data = np.array([[bb, tinggi_badan, umur, gender, tingkat_aktivitas, target_bb]])

# Menampilkan data yang dimasukkan
print("Data pengguna yang dimasukkan:", new_user_data)


In [ ]:
predicted_cluster = model.predict(new_user_data)

# Ambil cluster dengan probabilitas tertinggi
predicted_cluster = np.argmax(predicted_cluster, axis=1)
print("Cluster yang diprediksi:", predicted_cluster[0])

In [ ]:
cluster_user = predicted_cluster[0]

# Filter dataset resep berdasarkan cluster_user
filtered_resep = df_resep[df_resep['cluster'] == cluster_user]

# Acak data resep agar distribusi lebih bervariasi
filtered_resep = filtered_resep.sample(frac=1, random_state=42).reset_index(drop=True)

# Ambil 3 resep untuk masing-masing waktu makan
resep_pagi = filtered_resep.head(3)  # Resep pertama untuk pagi
resep_siang = filtered_resep.iloc[3:6]  # Resep berikutnya untuk siang
resep_malam = filtered_resep.iloc[6:9]  # Resep berikutnya untuk malam

# Jika data kurang dari 9, ambil sesuai yang tersedia
if len(filtered_resep) < 9:
    resep_pagi = filtered_resep.iloc[:min(3, len(filtered_resep))]
    resep_siang = filtered_resep.iloc[min(3, len(filtered_resep)):min(6, len(filtered_resep))]
    resep_malam = filtered_resep.iloc[min(6, len(filtered_resep)):min(9, len(filtered_resep))]


In [17]:
# Gabungkan semua resep untuk pagi, siang, malam menggunakan pd.concat
all_resep = pd.concat([resep_pagi, resep_siang, resep_malam], ignore_index=True)

# Cek apakah ada duplikasi resep
if len(all_resep) != len(set(all_resep['nama_makanan'])):  # Jika ada duplikasi
    print("Terjadi duplikasi resep. Menyaring ulang data...")

    # Proses untuk menyaring ulang data berdasarkan nama_makanan
    unique_resep = all_resep.drop_duplicates(subset=['nama_makanan'])  # Menghilangkan duplikasi berdasarkan 'nama_makanan'

    print("Data setelah penyaringan ulang (tanpa duplikasi):")

    # Pisahkan kembali data berdasarkan waktu makan (pagi, siang, malam)
    resep_pagi_unique = unique_resep[unique_resep['jenis'] == 'Pokok']
    resep_siang_unique = unique_resep[unique_resep['jenis'].isin(['Lauk', 'Sayuran'])]
    resep_malam_unique = unique_resep[unique_resep['jenis'] == 'Cemilan']

    # Tampilkan hasil setelah penyaringan ulang
    print("\nRekomendasi Pagi (setelah penyaringan):")
    for _, row in resep_pagi_unique.iterrows():
        print(f"Menu: {row['nama_makanan']}")
        print(f"Kalori: {row['kalori']}")
        # Menampilkan gambar jika ada link gambar
        if pd.notna(row['image']):
            display(Image(url=row['image']))  # Menampilkan gambar
        print("Bahan:")
        for i in range(1, 16):  # Mengambil bahan1 sampai bahan15
            bahan = row[f'bahan{i}']
            jumlah = row[f'jumlah{i}']
            if pd.notna(bahan) and pd.notna(jumlah):
                print(f"- {bahan}: {jumlah}")
        print("Cara Masak:", row['cara_masak'])
        print("-" * 20)  # Pemisah antar menu

    print("\nRekomendasi Siang (setelah penyaringan):")
    for _, row in resep_siang_unique.iterrows():
        print(f"Menu: {row['nama_makanan']}")
        print(f"Kalori: {row['kalori']}")
        if pd.notna(row['image']):
            display(Image(url=row['image']))
        print("Bahan:")
        for i in range(1, 16):
            bahan = row[f'bahan{i}']
            jumlah = row[f'jumlah{i}']
            if pd.notna(bahan) and pd.notna(jumlah):
                print(f"- {bahan}: {jumlah}")
        print("Cara Masak:", row['cara_masak'])
        print("-" * 20)

    print("\nRekomendasi Malam (setelah penyaringan):")
    for _, row in resep_malam_unique.iterrows():
        print(f"Menu: {row['nama_makanan']}")
        print(f"Kalori: {row['kalori']}")
        if pd.notna(row['image']):
            display(Image(url=row['image']))
        print("Bahan:")
        for i in range(1, 16):
            bahan = row[f'bahan{i}']
            jumlah = row[f'jumlah{i}']
            if pd.notna(bahan) and pd.notna(jumlah):
                print(f"- {bahan}: {jumlah}")
        print("Cara Masak:", row['cara_masak'])
        print("-" * 20)

else:
    # Tampilkan hasil rekomendasi tanpa duplikasi
    print("Rekomendasi Pagi:")
    for _, row in resep_pagi.iterrows():
        print(f"Menu: {row['nama_makanan']}")
        print(f"Kalori: {row['kalori']}")
        if pd.notna(row['image']):
            display(Image(url=row['image']))
        print("Bahan:")
        for i in range(1, 16):
            bahan = row[f'bahan{i}']
            jumlah = row[f'jumlah{i}']
            if pd.notna(bahan) and pd.notna(jumlah):
                print(f"- {bahan}: {jumlah}")
        print("Cara Masak:", row['cara_masak'])
        print("-" * 20)

    print("\nRekomendasi Siang:")
    for _, row in resep_siang.iterrows():
        print(f"Menu: {row['nama_makanan']}")
        print(f"Kalori: {row['kalori']}")
        if pd.notna(row['image']):
            display(Image(url=row['image']))
        print("Bahan:")
        for i in range(1, 16):
            bahan = row[f'bahan{i}']
            jumlah = row[f'jumlah{i}']
            if pd.notna(bahan) and pd.notna(jumlah):
                print(f"- {bahan}: {jumlah}")
        print("Cara Masak:", row['cara_masak'])
        print("-" * 20)

    print("\nRekomendasi Malam:")
    for _, row in resep_malam.iterrows():
        print(f"Menu: {row['nama_makanan']}")
        print(f"Kalori: {row['kalori']}")
        if pd.notna(row['image']):
            display(Image(url=row['image']))
        print("Bahan:")
        for i in range(1, 16):
            bahan = row[f'bahan{i}']
            jumlah = row[f'jumlah{i}']
            if pd.notna(bahan) and pd.notna(jumlah):
                print(f"- {bahan}: {jumlah}")
        print("Cara Masak:", row['cara_masak'])
        print("-" * 20)


Rekomendasi Pagi:
Menu: ayam sambal ijo kecombrang
Kalori: 949.0


Bahan:
- ayam: 1/2 ekor
- cabai hijau keriting: 100 gram
- cabai rawit: 15 buah
- bawang merah: 10 biji
- bawang putih: 10 biji
- tomat hijau: 1 buah
- kecombrang: 2 biji
- air lemon: 1 sdm
- penyedap rasa: secukupnya
- air: 500 ml
Cara Masak: 1. potong ayam menjadi 6 bagian atau seperlunya kemudian Marinasi 30 menit pakai teknik masing² 2.goreng ayam yang telah dimarinasi sampai ½ matang 3.haluskan Bahan² bumbu kecuali Kecombrang kemudian tumis sampai matang 4.setelah bumbu matang, masukkan ayam dan air serta tambahkan penyedap rasa. 5.jika sudah sampai setengah kering, masukkan irisan kecombrang. Masak sampai air susut dan kering 6. makanan siap disajikan\
--------------------
Menu: mie goreng bakso
Kalori: 602.0


Bahan:
- mie urai: 70 gram
- kubis: 1 lembar
- bakso: 4 butir
- telur: 1 butir
- bawang merah: 2 siung
- bawang putih: 1/2 siung
- cabai: secukupnya
- lada: secukupnya
- garam: 1/3 sdt
- kecap manis: 2 sdm
Cara Masak: 1.rebus mie kering hingga lemas dan tiriskan. Haluskan bumbu halus. 2.panaskan wajan dengan 2 sdm minyak goreng. Buat dulu telur orak-arik, lalu sisihkan. Tuang sedikit minyak, tumis bumbu halus hingga harum. Masukkan semua bahan lain. Beri kecap, lada garam, tes rasa. Aduk hingga hingga bumbu merata. 3. segera sajikan  
--------------------
Menu: sandwich roti gandum
Kalori: 555.0


Bahan:
- roti gandum: 2 lembar
- sosis: 2 buah
- margarin: 1/2 sdm
- saus sambal: secukupnya
- mayonaise: secukupnya
Cara Masak: 1.Masukan margarin ke teflon setelah panas, panggang sosis sampai agak kering (atau sesuai selera) 2.Gepengkan roti gandum, lalu masukan keju dan sosis yang sudah di panggang, lalu gulung 3.Roti yang sudah digulung di panggang lagi di teflon hingga kecoklatan 4.sandwich roti gandum yg crunchy siap di santap
--------------------

Rekomendasi Siang:
Menu: balado ayam
Kalori: 924.6


Bahan:
- ayam ungkep: 6 potong
- minyak: 2 sdm
- air: 30 ml
- garam: 1/4 sdt
- gula pasir: 1/4 sdt
- bumbu balado instan: 50 gram
- daun salam: 1 lembar
- daun jeruk: 1 lembar
- bawang putih: 1,5 siung
- bawang merah: 2 siung
Cara Masak: 1. rebus air, masukkan ayam bersama air asam Jawa, garam, lada bubuk & daun salam. Ungkep sampai matang. Angkat. Dan tiriskan. Goreng ayam sampai kuning kecoklatan, angkat, tiriskan 2.siapakan bumbu2nya.
Tumis bawang merah dan bawang putih hingga harum, masukkan bumbu balado serbaguna instant dan juga daun salam daun serehnya hingga layu, masukkan ayam yang sudah digoreng, 3.Lalu aduk rata, tambahkan air, masak hingga air menyusut 4. angkat dan siap sajikan
--------------------
Menu: monday chicken salad
Kalori: 607.0


Bahan:
- daging ayam fillet: 300 gram
- telur: 2 butir
- pear: 1/2 buah
- tomat ceri: 200 gram
- timun: 1 pcs
- alpukat: 1 buah
- lobak merah: 3 buah
- rucola: 1 ikat
- selada romaine: 4 buah
- bawang putih: 2 siung
- mustard: secukupnya
- garam: secukupnya
- olive oil: secukupnya
- cuka: secukupnya
- lada hitam: secukupnya
Cara Masak: 1.siapkan bumbu untuk marinasi ayam -campurkan 1SDM mustard, 3sdm olive oil, 1SDT garam, lada, oregano secukupnya) bersihkan ayam terlebih dulu dan masukkan daging ayam kedalam bumbu dan biarkan 5-10menit agar bumbu lebih meresap) Dibagian ini aku potong daging ayam kecil2 ya agar lebih mudah termarinasi. 2.setelah itu siapkan wajan untuk menggoreng, tidak perlu menambah minyak, cukup masukkan semua bumbu untuk me marinasi ayam untuk menggoreng ayam, jangan lupa untuk membalik ayam agar matang merata, dan tunggu hingga bumbunya terserap habis dan menyisakan minyaknya. 3.didihkan air untuk merebus telur, setelah air mendidih masukkan telur dan tunggu 7-8m

Bahan:
- tempe: 350 gram
- telur: 1 butir
- daun bawang: 1 batang
- tepung terigu: 3 sdm
- tepung tapioka: 2 sdm
- tepung roti: secukupnya
- bawang putih: 3 siung
Cara Masak: 1. Kukus tempe hingga empuk selama sekitar 15 menit. 2. Haluskan tempe, kemudian campurkan dengan bawang merah dan bawang putih yang sudah dihaluskan. 3. Tambahkan telur, lalu masukkan tepung terigu dan tepung tapioka secara bertahap. 4. Tambahkan garam, kaldu ayam, dan daun bawang cincang halus. 5. Masukkan adonan ke dalam wadah tahan panas dan kukus selama 30 menit. 6. Setelah dingin, potong nugget tempe sesuai selera. 7. Kocok telur, balur nugget tempe dengan tepung roti, dan goreng hingga matang.
--------------------

Rekomendasi Malam:
Menu: perkedel tahu air fryer
Kalori: 598.0


Bahan:
- tahu putih: 8 buah
- telur: 1 buah
- wortel: 1/2 buah
- bumbu dasar bawang: secukupnya
- daun bawang: 2 batang
- merica: secukupnya
- minyak kelapa: 1 sdm
Cara Masak: 1. aduk dan hancurkan semua bahan, bentuk bulatan. bila kurang menempel, bisa ditambah telur lagi. 2. semprot dengan minyak kelapa, tata di airfryer, panggang 120°C selama 15 menit.
--------------------
Menu: ayam masak nanas
Kalori: 866.0


Bahan:
- ayam: 1/2 ekor
- nanas madu mini: 1/2 buah
- garam: secukupnya
- penyedap rasa: secukupnya
- lada: secukupnya
- cabai bubuk: secukupnya
- saus tiram: secukupnya
- gula pasir: 1/2 sdt
- tepung maizena: 1 sdt
- air: secukupnya
Cara Masak: 1.siapkan 1/2 ekor ayam yg sudah dipotong, cuci bersih. Masukan wadah Tambahkan bumbu, dan saus tiram tutup, diamkan ± 30 Menit 2.siapkan potongan nanas, sisihkan.
Panaskan secukupnya minyak di wajan.
Masak ayam di kedua sisi. 3. masak hingga matang kecoklatan dan empuk, Angkat dan sisihkan. 4.di wajan yg sama masak nanas sebentar, lalu tambahkan ayam tadi 5.tambahkan saus tiram lg secukupnya, aduk rata.
Terakhir masukan larutan maizena.
Masak sebentar sampai mengental.

Angkat, letakan di piring dan siap disajikan
--------------------
Menu: ayam kuah angkak
Kalori: 740.0


Bahan:
- ayam kampung: 1 ekor
- angkak: 1 sdm
- jahe: 5 cm
- air: 2 liter
- garam: secukupnya
- gula: secukupnya
- merica: secukupnya
Cara Masak: 1. Rebus air dan ayam bersama jahe hingga mendidih dan mengeluarkan kaldunya. 2. Tambahkan angkak, gula, garam, dan merica. Rebus kembali hingga ayam empuk dengan api sedang. 3. Angkat dan sajikan selagi hangat.
--------------------
